# Execute the Graph-Based Spatial Cross-Validation experiments from ICMLA21 Paper

## 1 - Initialize libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
from weka.core import jvm
from src import utils
from src.pipeline import Pipeline
from src.visualization.performance import VizMetrics
from src.visualization.dependence import VizDependence

/home/tpinho/miniconda3/envs/RegGBSCV/lib/python3.8/site-packages/pysal/explore/segregation/network/network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(


## 2 - Initialize loggers

In [2]:
utils.initialize_coloredlog()
utils.initialize_rich_tracerback()
utils.initialize_logging()

## 3 - Initialize working path and enviromental variables

In [3]:
# Project path
project_dir = os.path.abspath('')[:-5]
# Load enviromental variables
env_var = utils.load_env_variables(project_dir)

## 4 - Set pipeline switchers, the default is to set True to all processes

In [4]:
# Set pipeline switchers
SWITCHERS = {
    "scv": False,
    "fs": False,
    "train": True,
    "predict": True,
    "evaluate": False,
}

# 4 - List all datasets

In [5]:
brazil_removed_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_north",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_northeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_south",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_southeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_centerwest"]

brazil_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.1",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.2",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.3",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.4",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.5",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.6",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.7",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.8",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.9"]
    

brazil_geoeconomical_regions = ["Brazil_Election_2018_removed_AMAZONIA",
                                "Brazil_Election_2018_removed_NORDESTE",
                                "Brazil_Election_2018_removed_CENTRO_SUL"]

us_datasets = ["US_Corn_Yield_2016", "US_Wheat_Yield_2014"]
just_for_test = ["Brazil_Election_2018_removed_CENTRO_SUL"]

## 6 - Runs the pipeline for each method
OBS: The results and files generated from the pipeline execution will be in the created folder Results in the data directory

### 6.1 Traditional SCV
OBS: We the the paramenter fast True so the semivariogram calculation step that can take 24h is skipped. We calculate the removing buffer by considering the 27 n-degree neighborhood as stated in the paper 

In [ ]:
dataset_list = brazil_datasets
fs_method = "CFS"
ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "RF", "MLP", "SVM"]


if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.start()
# Set paths
meshblocks_filename = "meshblocks.shp"
meshblocks_idx = "code_muni"
data_idx = "INDEX"
env_var["root_path"] = "/exp/tpinho/Datasets/"
for dataset in dataset_list:
    data_path = os.path.join(env_var["root_path"], dataset, "data.csv")
    
    meshblocks_path = os.path.join(env_var["root_path"], dataset, "meshblocks", meshblocks_filename)
    # Load data
    data = pd.read_csv(data_path, index_col=data_idx, low_memory=False)
    try:
        data.drop(columns=["[GEO]_LATITUDE", "[GEO]_LONGITUDE"], inplace=True)
    except KeyError:
        pass
    meshblocks = gpd.read_file(meshblocks_path)
    # Set meshblocks  index
    meshblocks.set_index(meshblocks_idx, inplace=True)
    for ml_method in ml_methods:
        # Run pipeline
        TraditionalSCV = Pipeline(
            root_path=os.path.join(env_var["root_path"], dataset),
            data=data,
            meshblocks=meshblocks,
            index_col="INDEX",
            fold_col="INDEX_FOLDS",
            target_col="TARGET",
            scv_method="TraditionalSCV",
            fs_method=fs_method,
            ml_method=ml_method,
            switchers=SWITCHERS
        )

        print(f"Running the Traditional SCV approach for dataset: {dataset} ML Method = {ml_method}")
        TraditionalSCV.run()
if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.stop()

Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.1 ML Method = KNN


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 43.53it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.1 ML Method = OLS


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 58.33it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.1 ML Method = Lasso


...

Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:01<00:00, 16.49it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.1 ML Method = LGBM


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 29.28it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.1 ML Method = RF


Training model:  85%|██████████████████████████████████████████████████████████████████████████████████▋              | 23/27 [11:22<01:58, 29.69s/it]
